In [2]:
import mlflow

# Yerel MLflow server adresini belirtiyoruz
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Deneyin adını belirtiyoruz (uzun kodda zaten ayarlanmıştı)
mlflow.set_experiment("Tıbbi Mortalite Tahmini")
print("MLflow Sunucuya Bağlandı.")

MLflow Sunucuya Bağlandı.


In [6]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install interpret

     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ------- -------------------------------- 0.8/4.0 MB 5.6 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 6.2 MB/s eta 0:00:01
     ------------------------------------ --- 3.7/4.0 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------- 4.0/4.0 MB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.6 MB 4.2 MB/s eta 0:00:04
   --- ------------------------------------ 1.6/16.6 MB 4.0 MB/s eta 0:00:04
   ------ --------------------------------- 2.6/16.6 MB 4.2 MB/s eta 0:00:04
   -------- ------------------------------- 3.4/16.6 MB 4.2 MB/s eta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.33.1 which is incompatible.


In [14]:
pip install xgboost interpret tensorflow scikit-learn pandas imblearn

   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.8/331.9 MB 4.8 MB/s eta 0:01:09
   ---------------------------------------- 2.1/331.9 MB 6.2 MB/s eta 0:00:54
   ---------------------------------------- 3.4/331.9 MB 6.3 MB/s eta 0:00:53
    --------------------------------------- 4.7/331.9 MB 6.1 MB/s eta 0:00:54
    --------------------------------------- 5.8/331.9 MB 6.0 MB/s eta 0:00:55
    --------------------------------------- 7.1/331.9 MB 6.1 MB/s eta 0:00:54
   - -------------------------------------- 8.7/331.9 MB 6.2 MB/s eta 0:00:53
   - -------------------------------------- 10.0/331.9 MB 6.1 MB/s eta 0:00:53
   - -------------------------------------- 11.0/331.9 MB 6.0 MB/s eta 0:00:54
   - -------------------------------------- 11.8/331.9 MB 5.9 MB/s eta 0:00:55
   - -------------------------------------- 12.8/331.9 MB 5.7 MB/s eta 0:00:56
   - -------------------------------------- 13.6/331.9 MB 5.6 MB/s 

In [3]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from interpret.glassbox import ExplainableBoostingClassifier
import tensorflow as tf
from tensorflow.keras import layers, Sequential

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    average_precision_score, matthews_corrcoef
)

# Yerel MLflow server adresini belirtiyoruz
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Tıbbi Mortalite Tahmini") 

# MLflow'a loglama yapan fonksiyon (Tekrardan tanımlandı)
def evaluate_clf_and_log(y_true, y_pred, y_proba):
    metrics_dict = {
        "accuracy": round(accuracy_score(y_true, y_pred), 3),
        "precision": round(precision_score(y_true, y_pred, zero_division=0), 3),
        "recall": round(recall_score(y_true, y_pred, zero_division=0), 3),
        "f1_score": round(f1_score(y_true, y_pred, zero_division=0), 3),
        "roc_auc": round(roc_auc_score(y_true, y_proba), 3),
        "pr_auc": round(average_precision_score(y_true, y_proba), 3),
        "mcc": round(matthews_corrcoef(y_true, y_pred), 3)
    }
    print(f"ROC-AUC: {metrics_dict['roc_auc']}, Recall: {metrics_dict['recall']}")
    print("\nClassification Report:\n", classification_report(y_true, y_pred, zero_division=0))
    mlflow.log_metrics(metrics_dict)
    return metrics_dict

# ==============================================================================
# 1. VERİ YÜKLEME VE HAZIRLAMA (SENİN CSV DOSYAN)
# ==============================================================================

# Gerçek CSV dosyanızı okuyoruz
df = pd.read_csv("synthetic_medical_data_cleaned.csv") 

# 1️⃣ Sayısal sütunları median ile doldur
num_cols_to_impute = df.select_dtypes(include=np.number).columns.drop('Dead', errors='ignore')
df[num_cols_to_impute] = df[num_cols_to_impute].fillna(df[num_cols_to_impute].median())

# 2️⃣ Kategorik sütunları "Missing" ile doldur ve kategorik tipe çevir
cat_cols = df.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    if col in df.columns:
        # Önce kategorik tipe dönüştür
        df[col] = df[col].astype('category')
        # Sonra 'Missing' kategorisini ekle ve NaN'ları doldur
        if 'Missing' not in df[col].cat.categories:
            df[col] = df[col].cat.add_categories('Missing')
        df[col] = df[col].fillna("Missing")

# 3️⃣ One-hot encoding
df_encoded = pd.get_dummies(df, drop_first=True)

# 4️⃣ Bağımsız değişkenler (X) ve hedef (y)
X = df_encoded.drop("Dead", axis=1)
y = df_encoded["Dead"]

# 5️⃣ Veriyi train/test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("\n✅ Veri Hazırlandı!")
print(f"Train Boyutu: {X_train.shape}, Test Boyutu: {X_test.shape}")
print(f"X'in ilk 5 sütunu: {X.columns[:5].tolist()}")


✅ Veri Hazırlandı!
Train Boyutu: (485, 75), Test Boyutu: (122, 75)
X'in ilk 5 sütunu: ['Age', 'Age_in_Diagnosis', 'Gender', 'Follow_Up_Duration_Month', 'Disease_Duration_Month']


In [4]:
# Model 1: Logistic Regression (Scaled Pipeline)
print("--- 1. Logistic Regression (Scaled Pipeline) ---")
with mlflow.start_run(run_name="LogisticRegression_Scaled_Baseline"):

    lr_params = {
        'max_iter': 2000,
        'class_weight': 'balanced',
        'random_state': 42
    }
    mlflow.log_params(lr_params)
    mlflow.log_param("preprocessing", "StandardScaler")

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('log_reg', LogisticRegression(**lr_params))
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]

    # Metrikleri logla ve modeli kaydet
    evaluate_clf_and_log(y_test, y_pred, y_proba)
    mlflow.sklearn.log_model(pipe, "model")

2025/11/16 23:33:31 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



--- 1. Logistic Regression (Scaled Pipeline) ---


2025/11/16 23:33:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


ROC-AUC: 0.386, Recall: 0.1

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.77      0.83       112
           1       0.04      0.10      0.05        10

    accuracy                           0.71       122
   macro avg       0.47      0.43      0.44       122
weighted avg       0.83      0.71      0.77       122



2025/11/16 23:33:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression_Scaled_Baseline at: http://127.0.0.1:5000/#/experiments/362671437737102849/runs/d8bb282f564545b2bb55365155c6c768
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/362671437737102849


In [5]:
# Model 2: RandomForest (Class Weighted)
print("\n--- 2. RandomForest (Class Weighted) ---")
with mlflow.start_run(run_name="RandomForest_ClassWeight"):

    rf_params = {
        'n_estimators': 300,
        'max_depth': None,
        'class_weight': 'balanced', 
        'random_state': 42
    }
    mlflow.log_params(rf_params)

    rf = RandomForestClassifier(**rf_params)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    y_proba = rf.predict_proba(X_test)[:, 1]

    # Metrikleri logla
    evaluate_clf_and_log(y_test, y_pred, y_proba)
    mlflow.sklearn.log_model(rf, "model")


--- 2. RandomForest (Class Weighted) ---


2025/11/16 23:46:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


ROC-AUC: 0.434, Recall: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96       112
           1       0.00      0.00      0.00        10

    accuracy                           0.92       122
   macro avg       0.46      0.50      0.48       122
weighted avg       0.84      0.92      0.88       122



2025/11/16 23:46:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForest_ClassWeight at: http://127.0.0.1:5000/#/experiments/362671437737102849/runs/8eb7bf609e9440edb60d96c9522ea880
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/362671437737102849


In [6]:
# Model 3: XGBoost (5-Fold CV - Avg)
print("\n--- 3. XGBoost (5-Fold CV - Avg) ---")

metrics_keys = ["accuracy", "precision", "recall", "f1_score", "roc_auc", "pr_auc", "mcc"]
all_metrics = {key: [] for key in metrics_keys}

with mlflow.start_run(run_name="XGBoost_5_Fold_CV") as parent_run:

    xgb_cv_params = {
        'n_estimators': 400,
        'max_depth': 5,
        'learning_rate': 0.05,
        'scale_pos_weight': 10,
        'random_state': 42
    }
    mlflow.log_params(xgb_cv_params)
    mlflow.log_param("n_splits", 5)
    mlflow.log_param("eval_threshold", 0.3) 

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold = 1
    
    # Tüm veri setini kullanıyoruz (X, y), K-Fold CV içinde ayırma yapılıyor
    for train_idx, val_idx in skf.split(X, y): 
        with mlflow.start_run(run_name=f"XGB_Fold_{fold}", nested=True):
            X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = XGBClassifier(**xgb_cv_params)
            model.fit(X_tr, y_tr)

            # Eşik 0.3 ile değerlendiriyoruz (Recall odaklı)
            y_proba = model.predict_proba(X_val)[:, 1]
            y_pred = (y_proba >= 0.3).astype(int) 

            fold_metrics = evaluate_clf_and_log(y_val, y_pred, y_proba)
            
            for key in metrics_keys:
                all_metrics[key].append(fold_metrics[key])

        fold += 1

    # Ana çalışmaya (Parent Run) ortalama metrikleri logla
    print("\n========== 📊 XGB Ortalama Sonuçlar (5-Fold) ==========")
    avg_metrics = {}
    for m in metrics_keys:
        mean_val = np.mean(all_metrics[m])
        std_val = np.std(all_metrics[m])
        print(f"{m.upper():<10}: {mean_val:.3f} ± {std_val:.3f}")
        avg_metrics[f"avg_{m}"] = mean_val
        avg_metrics[f"std_{m}"] = std_val

    mlflow.log_metrics(avg_metrics)


--- 3. XGBoost (5-Fold CV - Avg) ---
ROC-AUC: 0.538, Recall: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94       112
           1       0.00      0.00      0.00        10

    accuracy                           0.89       122
   macro avg       0.46      0.49      0.47       122
weighted avg       0.84      0.89      0.87       122

🏃 View run XGB_Fold_1 at: http://127.0.0.1:5000/#/experiments/362671437737102849/runs/4aa6202224ac4317be8463f6f6e678b3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/362671437737102849
ROC-AUC: 0.609, Recall: 0.182

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.95      0.94       111
           1       0.29      0.18      0.22        11

    accuracy                           0.89       122
   macro avg       0.60      0.57      0.58       122
weighted avg       0.86      0.89      0.87       122

🏃 Vi

In [7]:
# Model 4: EBM (5-Fold CV - Avg)
print("\n--- 4. EBM (5-Fold CV - Avg) ---")

metrics_keys = ["accuracy", "precision", "recall", "f1_score", "roc_auc", "pr_auc", "mcc"]
all_metrics = {key: [] for key in metrics_keys}

with mlflow.start_run(run_name="EBM_5_Fold_CV") as parent_run:

    ebm_params = {
        'interactions': 10,
        'random_state': 42
    }
    mlflow.log_params(ebm_params)
    mlflow.log_param("n_splits", 5)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold = 1
    
    for train_idx, val_idx in skf.split(X, y):
        with mlflow.start_run(run_name=f"EBM_Fold_{fold}", nested=True):
            X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = ExplainableBoostingClassifier(**ebm_params)
            model.fit(X_tr, y_tr)

            y_proba = model.predict_proba(X_val)[:, 1]
            y_pred = (y_proba >= 0.5).astype(int) # Default eşik ile logluyoruz

            fold_metrics = evaluate_clf_and_log(y_val, y_pred, y_proba)

            for key in metrics_keys:
                all_metrics[key].append(fold_metrics[key])

        fold += 1

    # Ana çalışmaya ortalama metrikleri logla
    print("\n========== 📊 EBM Ortalama Sonuçlar (5-Fold) ==========")
    avg_metrics = {}
    for m in metrics_keys:
        mean_val = np.mean(all_metrics[m])
        std_val = np.std(all_metrics[m])
        print(f"{m.upper():<10}: {mean_val:.3f} ± {std_val:.3f}")
        avg_metrics[f"avg_{m}"] = mean_val
        avg_metrics[f"std_{m}"] = std_val

    mlflow.log_metrics(avg_metrics)


--- 4. EBM (5-Fold CV - Avg) ---
ROC-AUC: 0.563, Recall: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96       112
           1       0.00      0.00      0.00        10

    accuracy                           0.92       122
   macro avg       0.46      0.50      0.48       122
weighted avg       0.84      0.92      0.88       122

🏃 View run EBM_Fold_1 at: http://127.0.0.1:5000/#/experiments/362671437737102849/runs/fb1626389fbe46efb0e89aed3b199d9a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/362671437737102849
ROC-AUC: 0.546, Recall: 0.091

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95       111
           1       0.33      0.09      0.14        11

    accuracy                           0.90       122
   macro avg       0.62      0.54      0.55       122
weighted avg       0.86      0.90      0.88       122

🏃 View r

In [8]:
# Model 5: Neural Network (5-Fold CV - Avg)
print("\n--- 5. Neural Network (5-Fold CV - Avg) ---")

metrics_keys = ["accuracy", "precision", "recall", "f1_score", "roc_auc", "pr_auc", "mcc"]
all_metrics = {key: [] for key in metrics_keys}

with mlflow.start_run(run_name="NeuralNetwork_5_Fold_CV") as parent_run:

    nn_params = {
        'epochs': 40,
        'batch_size': 32,
        'layer1_units': 64,
        'layer2_units': 32,
        'optimizer': 'adam'
    }
    mlflow.log_params(nn_params)
    mlflow.log_param("n_splits", 5)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold = 1
    
    for train_idx, val_idx in skf.split(X, y):
        with mlflow.start_run(run_name=f"NN_Fold_{fold}", nested=True):

            X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = Sequential([
                layers.Input(shape=(X_tr.shape[1],)),
                layers.Dense(nn_params['layer1_units'], activation='relu'),
                layers.Dense(nn_params['layer2_units'], activation='relu'),
                layers.Dense(1, activation='sigmoid')
            ])
            model.compile(optimizer=nn_params['optimizer'], loss='binary_crossentropy',
                          metrics=[tf.keras.metrics.AUC(name='auc')])

            model.fit(X_tr, y_tr, epochs=nn_params['epochs'], batch_size=nn_params['batch_size'],
                      validation_split=0.2, verbose=0)

            y_proba = model.predict(X_val).ravel()
            y_pred = (y_proba >= 0.5).astype(int)

            fold_metrics = evaluate_clf_and_log(y_val, y_pred, y_proba)

            for key in metrics_keys:
                all_metrics[key].append(fold_metrics[key])

        fold += 1

    # Ana çalışmaya ortalama metrikleri logla
    print("\n========== 📊 NN Ortalama Sonuçlar (5-Fold) ==========")
    avg_metrics = {}
    for m in metrics_keys:
        mean_val = np.mean(all_metrics[m])
        std_val = np.std(all_metrics[m])
        print(f"{m.upper():<10}: {mean_val:.3f} ± {std_val:.3f}")
        avg_metrics[f"avg_{m}"] = mean_val
        avg_metrics[f"std_{m}"] = std_val

    mlflow.log_metrics(avg_metrics)


--- 5. Neural Network (5-Fold CV - Avg) ---
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
ROC-AUC: 0.419, Recall: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.99      0.95       112
           1       0.00      0.00      0.00        10

    accuracy                           0.91       122
   macro avg       0.46      0.50      0.48       122
weighted avg       0.84      0.91      0.87       122

🏃 View run NN_Fold_1 at: http://127.0.0.1:5000/#/experiments/362671437737102849/runs/e428a434418c43448b7f3942edb1a160
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/362671437737102849
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
ROC-AUC: 0.396, Recall: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.95      0.93       111
           1       0.00      0.00      0.00        11

    accuracy                           0.87       122
   macro avg       0.45      0.48